In [ ]:
!pip install --upgrade wfdb
!pip install --upgrade xlsxwriter
!pip install --upgrade tensorflow_model_optimization
!pip install keras-visualizer

In [ ]:
import pandas as pd
import numpy as np
import wfdb
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.callbacks import LambdaCallback
from keras.layers import Conv1D, Flatten, LSTM, Dense, Dropout, TimeDistributed
from tensorflow.keras.optimizers import Adam
import tensorflow_model_optimization as tfmot
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quant_apply = tfmot.quantization.keras.quantize_apply
quantize_scope = tfmot.quantization.keras.quantize_scope


LastValueQuantizer = tfmot.quantization.keras.quantizers.LastValueQuantizer
MovingAverageQuantizer = tfmot.quantization.keras.quantizers.MovingAverageQuantizer


In [ ]:
# from google.colab import drive
# drive.mount("/content/gdrive")


In [ ]:
# !unzip gdrive/MyDrive/mit-bih-arrhythmia-database-1.0.0.zip

**Loading the data**

In [ ]:
dataset = {'mitdb': 'ARR', 'nsrdb': 'NSR', 'chfdb': 'CHF'}
for db, dir in dataset.items():
  wfdb.dl_database(db, os.getcwd()+'/'+dir)
  wfdb.dl_files(db, os.getcwd()+'/'+dir, ['RECORDS'])

Generating record list for: 100
Generating record list for: 101
Generating record list for: 102
Generating record list for: 103
Generating record list for: 104
Generating record list for: 105
Generating record list for: 106
Generating record list for: 107
Generating record list for: 108
Generating record list for: 109
Generating record list for: 111
Generating record list for: 112
Generating record list for: 113
Generating record list for: 114
Generating record list for: 115
Generating record list for: 116
Generating record list for: 117
Generating record list for: 118
Generating record list for: 119
Generating record list for: 121
Generating record list for: 122
Generating record list for: 123
Generating record list for: 124
Generating record list for: 200
Generating record list for: 201
Generating record list for: 202
Generating record list for: 203
Generating record list for: 205
Generating record list for: 207
Generating record list for: 208
Generating record list for: 209
Generati

In [ ]:
data_path = 'ARR/'
records = np.loadtxt("ARR/RECORDS", dtype=str)

In [ ]:
#These are the beat classifications according to physiobank

invalid = [ "[", "!", "]", "x", "(", ")", "p", "t", "u", "`", "'", "^", "|", "~", "+", "s", "T", "*", "D", "=", '"', "@" ]

abnormal = [ "L", "R", "B", "A", "a", "J", "S", "V", "r", "F", "e", "j", "n", "E", "/", "f", "Q", "?" ]


**Processing the dataset**

In [ ]:
def classify_beat(symbol):
    if symbol in abnormal :
        return 1
    elif symbol == "N" or symbol == ".":
        return 0

In [ ]:
def get_sequence(signal, beat_loc, window_sec, fs):
    window_one_side = window_sec * fs
    beat_start = beat_loc - window_one_side
    beat_end = beat_loc + window_one_side
    if beat_end < signal.shape[0]:
        sequence = signal[beat_start:beat_end, 0]
        return sequence.reshape(1, -1, 1)
    else:
        return np.array([])

In [ ]:
all_sequences = []
all_labels = []
window_sec = 3
subject_map = []
for subject in records:
    record = wfdb.rdrecord(f'/content/ARR/{subject}')
    annotation = wfdb.rdann(f'/content/ARR/{subject}', 'atr')
    atr_symbol = annotation.symbol
    atr_sample = annotation.sample
    fs = record.fs
    scaler = StandardScaler()
    signal = scaler.fit_transform(record.p_signal)
    subject_labels = []
    for i, i_sample in enumerate(atr_sample):
        label = classify_beat(atr_symbol[i])
        sequence = get_sequence(signal, i_sample, window_sec, fs)
        if label is not None and sequence.size > 0:
            all_sequences.append(sequence)
            subject_labels.append(label)

    normal_percentage = sum(subject_labels) / len(subject_labels)
    subject_map.append({
        "subject": subject,
        "percentage": normal_percentage,
        "num_seq": len(subject_labels),
        "start": len(all_labels),
        "end": len(all_labels)+len(subject_labels)
    })
    all_labels.extend(subject_labels)


In [ ]:
subject_map = pd.DataFrame(subject_map)

In [ ]:
bins = [0, 0.2, 0.6, 1.0]
subject_map["bin"] = pd.cut(subject_map['percentage'], bins=bins, labels=False, include_lowest=True)

In [ ]:
train, validation = train_test_split(subject_map, test_size=0.25, stratify=subject_map["bin"], random_state=42)

In [ ]:
def build_dataset(df, all_sequences, all_labels):
    sequences = []
    labels = []
    for i, row in df.iterrows():
        start = int(row["start"])
        end = int(row["end"])
        sequences.extend(all_sequences[start:end])
        labels.extend(all_labels[start:end])
        
    return np.vstack(sequences), np.vstack(labels)

In [ ]:
X_train, y_train = build_dataset(train, all_sequences, all_labels)
X_val, y_val = build_dataset(validation, all_sequences, all_labels)

In [ ]:
# data = pd.DataFrame(X_val.reshape(X_val.shape[0], X_val.shape[1]))
# data['label'] = y_val
# data
# data.to_csv('/content/gdrive/MyDrive/val.csv')

In [ ]:
X_train.shape, y_train.shape

((82873, 2160, 1), (82873, 1))

#Default Quantization Classes



In [ ]:
class DefaultQuantizeConfigCNN(tfmot.quantization.keras.QuantizeConfig):
    # Configure how to quantize weights.
    def get_weights_and_quantizers(self, layer):
      return [(layer.kernel, LastValueQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False))]

    # Configure how to quantize activations.
    def get_activations_and_quantizers(self, layer):
      return [(layer.activation, MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
      # Add this line for each item returned in `get_weights_and_quantizers`
      # , in the same order
      layer.kernel = quantize_weights[0]

    def set_quantize_activations(self, layer, quantize_activations):
      # Add this line for each item returned in `get_activations_and_quantizers`
      # , in the same order.
      layer._activation = quantize_activations[0]

    # Configure how to quantize outputs (may be equivalent to activations).
    def get_output_quantizers(self, layer):
      return []

    def get_config(self):
      return {}
class DefaultQuantizeConfigCNNLSTM(tfmot.quantization.keras.QuantizeConfig):
    # Configure how to quantize weights.
    def get_weights_and_quantizers(self, layer):
      return [(layer.layer.kernel, LastValueQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False))]

    # Configure how to quantize activations.
    def get_activations_and_quantizers(self, layer):
      return [(layer.layer.activation, MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
      # Add this line for each item returned in `get_weights_and_quantizers`
      # , in the same order
      layer.kernel = quantize_weights[0]

    def set_quantize_activations(self, layer, quantize_activations):
      # Add this line for each item returned in `get_activations_and_quantizers`
      # , in the same order.
      layer._activation = quantize_activations[0]

    # Configure how to quantize outputs (may be equivalent to activations).
    def get_output_quantizers(self, layer):
      return []

    def get_config(self):
      return {}
# class DefaultQuantizeFlattenConfigCNNLSTM(tfmot.quantization.keras.QuantizeConfig):
#     # Configure how to quantize weights.
#     def get_weights_and_quantizers(self, layer):
#       return [(layer.layer.kernel, LastValueQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False))]

#     # Configure how to quantize activations.
#     def get_activations_and_quantizers(self, layer):
#       return [(layer.layer.activation, MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]

#     def set_quantize_weights(self, layer, quantize_weights):
#       # Add this line for each item returned in `get_weights_and_quantizers`
#       # , in the same order
#       layer.kernel = quantize_weights[0]

#     def set_quantize_activations(self, layer, quantize_activations):
#       # Add this line for each item returned in `get_activations_and_quantizers`
#       # , in the same order.
#       layer._activation = quantize_activations[0]

#     # Configure how to quantize outputs (may be equivalent to activations).
#     def get_output_quantizers(self, layer):
#       return []

#     def get_config(self):
#       return {}
class DefaultQuantizeLSTMConfigCNNLSTM(tfmot.quantization.keras.QuantizeConfig):
    # Configure how to quantize weights.
    def get_weights_and_quantizers(self, layer):
      return [(layer.cell.kernel, LastValueQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False))]

    # Configure how to quantize activations.
    def get_activations_and_quantizers(self, layer):
      return [(layer.activation, MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
      # Add this line for each item returned in `get_weights_and_quantizers`
      # , in the same order
      layer.kernel = quantize_weights[0]

    def set_quantize_activations(self, layer, quantize_activations):
      # Add this line for each item returned in `get_activations_and_quantizers`
      # , in the same order.
      layer._activation = quantize_activations[0]

    # Configure how to quantize outputs (may be equivalent to activations).
    def get_output_quantizers(self, layer):
      return []

    def get_config(self):
      return {}

#CNN Model 

In [ ]:
sequence_size = X_train.shape[1]
n_features = 1

cnn_model = Sequential([
    quantize_annotate_layer(Conv1D(
        filters=8,
        kernel_size=4,
        strides=1,
        input_shape=(sequence_size, n_features),
        padding="same",
        activation="relu"
    ), DefaultQuantizeConfigCNN()),
    quantize_annotate_layer(Flatten()),
    quantize_annotate_layer(Dropout(0.5)),
    quantize_annotate_layer(Dense(
        1,
        activation="sigmoid",
        name="output",
    ))
])

optimizer = Adam(learning_rate=0.001)
# Compiling the model
cnn_model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy"]
)
cnn_model.summary()
cnn_model.save('cnn_model.h5')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_annotate_4 (Quantiz (None, 2160, 8)           40        
_________________________________________________________________
quantize_annotate_5 (Quantiz (None, 17280)             0         
_________________________________________________________________
quantize_annotate_6 (Quantiz (None, 17280)             0         
_________________________________________________________________
quantize_annotate_7 (Quantiz (None, 1)                 17281     
Total params: 17,321
Trainable params: 17,321
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# def get_flopsCNN(model_h5_path):
#     session = tf.compat.v1.Session()
#     graph = tf.compat.v1.get_default_graph()


#     with graph.as_default():
#         with session.as_default():
#             with quantize_scope({'DefaultQuantizeConfigCNN': DefaultQuantizeConfigCNN}):
#               model = tf.keras.models.load_model(model_h5_path)

#               run_meta = tf.compat.v1.RunMetadata()
#               opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
#               flops = tf.compat.v1.profiler.profile(graph=graph,
#                                                     run_meta=run_meta, cmd='op', options=opts)

#               return flops.total_float_ops

In [ ]:
# print_weights = LambdaCallback(on_epoch_end=lambda batch, logs: print(cnn_model.layers[0].get_weights()))
hist_cnn = cnn_model.fit(
    X_train, 
    y_train, 
    batch_size=128,
    epochs=15,
    validation_data=(X_val, y_val)
)

Epoch 1/15
 45/648 [=>............................] - ETA: 44s - loss: 0.4201 - accuracy: 0.8151

KeyboardInterrupt: ignored

In [ ]:
# cnn_model.load_weights(pretrained_weights) # optional but recommended for model accuracy
with quantize_scope(
    {'DefaultQuantizeConfigCNN': DefaultQuantizeConfigCNN}):
    cnn_quant_aware = tfmot.quantization.keras.quantize_apply(cnn_model)
cnn_quant_aware.summary()
cnn_quant_aware.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy"]
)
cnn_quant_aware.fit(
    X_train, 
    y_train, 
    batch_size=128,
    epochs=15,
    validation_data=(X_val, y_val)
)

#CNN + LSTM

In [ ]:
sequence_size = X_train.shape[1]
n_features = 1 
n_subsequences = 4
subsequence_size = int(sequence_size / n_subsequences)

# Reshaping to be (samples, subsequences, sequence, feature)
X_train = X_train.reshape(-1, n_subsequences, subsequence_size, n_features)
X_val = X_val.reshape(-1, n_subsequences, subsequence_size, n_features)

In [ ]:
cnn_lstm_model = Sequential([
    quantize_annotate_layer(TimeDistributed(
        Conv1D(
            filters=4,
            kernel_size=4,
            strides=1,
            padding="same",
            activation="relu"
        ), 
        input_shape=(n_subsequences, subsequence_size, n_features)
    ), DefaultQuantizeConfigCNNLSTM()),
    TimeDistributed(quantize_annotate_layer(Flatten())),
    quantize_annotate_layer(LSTM(units=2), DefaultQuantizeLSTMConfigCNNLSTM()),
    quantize_annotate_layer(Dense(
        1,
        activation="sigmoid",
        name="output",
    ))
])

optimizer = Adam(learning_rate=0.001)
# Compiling the model
cnn_lstm_model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy"]
)
cnn_lstm_model.summary()
cnn_lstm_model.save('cnn_lstm_model.h5')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_annotate_4 (Quantiz (None, 4, 540, 4)         20        
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 2160)           0         
_________________________________________________________________
quantize_annotate_6 (Quantiz (None, 2)                 17304     
_________________________________________________________________
quantize_annotate_7 (Quantiz (None, 1)                 3         
Total params: 17,327
Trainable params: 17,327
Non-trainable params: 0
_________________________________________________________________


In [ ]:
train_params = {
    "batch_size": 128,
    "epochs": 15,
    "verbose": 1,
    "validation_data": (X_val, y_val),
}

# history_cnn_lstm = cnn_lstm_model.fit(X_train, y_train, **train_params)

In [ ]:
# quant_aware_model = tfmot.quantization.keras.quantize_apply(cnn_lstm_model)
with quantize_scope(
    {'DefaultQuantizeConfigCNNLSTM': DefaultQuantizeConfigCNNLSTM,
     'DefaultQuantizeLSTMConfigCNNLSTM': DefaultQuantizeLSTMConfigCNNLSTM}):
    cnn_lstm_quant_aware = tfmot.quantization.keras.quantize_apply(cnn_lstm_model)
cnn_lstm_quant_aware.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy"]
)
history_cnn_lstm = cnn_lstm_quant_aware.fit(X_train, y_train, **train_params)
cnn_lstm_quant_aware.summary()

Epoch 1/15
648/648 [==============================] - 61s 91ms/step - loss: 0.3033 - accuracy: 0.8764 - val_loss: 0.5271 - val_accuracy: 0.7746
Epoch 2/15
648/648 [==============================] - 58s 89ms/step - loss: 0.1833 - accuracy: 0.9466 - val_loss: 0.5304 - val_accuracy: 0.8142
Epoch 3/15
648/648 [==============================] - 57s 88ms/step - loss: 0.1462 - accuracy: 0.9585 - val_loss: 0.5475 - val_accuracy: 0.7998
Epoch 4/15
648/648 [==============================] - 57s 88ms/step - loss: 0.1265 - accuracy: 0.9641 - val_loss: 0.5813 - val_accuracy: 0.8053
Epoch 5/15
648/648 [==============================] - 58s 89ms/step - loss: 0.1133 - accuracy: 0.9682 - val_loss: 0.5692 - val_accuracy: 0.8097
Epoch 6/15
648/648 [==============================] - 58s 90ms/step - loss: 0.1014 - accuracy: 0.9713 - val_loss: 0.5757 - val_accuracy: 0.8173
Epoch 7/15
648/648 [==============================] - 57s 89ms/step - loss: 0.0917 - accuracy: 0.9738 - val_loss: 0.5766 - val_accuracy: